# Challenge 2: Sentiment Analysis

In this challenge we will learn sentiment analysis and practice performing sentiment analysis on Twitter tweets. 

## Introduction

Sentiment analysis is to *systematically identify, extract, quantify, and study affective states and subjective information* based on texts ([reference](https://en.wikipedia.org/wiki/Sentiment_analysis)). In simple words, it's to understand whether a person is happy or unhappy in producing the piece of text. Why we (or rather, companies) care about sentiment in texts? It's because by understanding the sentiments in texts, we will be able to know if our customers are happy or unhappy about our products and services. If they are unhappy, the subsequent action is to figure out what have caused the unhappiness and make improvements.

Basic sentiment analysis only understands the *positive* or *negative* (sometimes *neutral* too) polarities of the sentiment. More advanced sentiment analysis will also consider dimensions such as agreement, subjectivity, confidence, irony, and so on. In this challenge we will conduct the basic positive vs negative sentiment analysis based on real Twitter tweets.

NLTK comes with a [sentiment analysis package](https://www.nltk.org/api/nltk.sentiment.html). This package is great for dummies to perform sentiment analysis because it requires only the textual data to make predictions. For example:

```python
>>> from nltk.sentiment.vader import SentimentIntensityAnalyzer
>>> txt = "Ironhack is a Global Tech School ranked num 2 worldwide.   Our mission is to help people transform their careers and join a thriving community of tech professionals that love what they do."
>>> analyzer = SentimentIntensityAnalyzer()
>>> analyzer.polarity_scores(txt)
{'neg': 0.0, 'neu': 0.741, 'pos': 0.259, 'compound': 0.8442}
```

In this challenge, however, you will not use NLTK's sentiment analysis package because in your Machine Learning training in the past 2 weeks you have learned how to make predictions more accurate than that. The [tweets data](https://www.kaggle.com/kazanova/sentiment140) we will be using today are already coded for the positive/negative sentiment. You will be able to use the Naïve Bayes classifier you learned in the lesson to predict the sentiment of tweets based on the labels.

## Conducting Sentiment Analysis

### Loading and Exploring Data

The dataset we'll be using today is located on Kaggle (https://www.kaggle.com/kazanova/sentiment140). Once you have downloaded and imported the dataset, it you will need to define the columns names: df.columns = ['target','id','date','flag','user','text']

*Notes:* 

* The dataset is huuuuge (1.6m tweets). When you develop your data analysis codes, you can sample a subset of the data (e.g. 20k records) so that you will save a lot of time when you test your codes.

In [1]:
import pandas as pd
import numpy as np

In [2]:
new_skip = np.linspace(1, 1599999, 1569999)

In [3]:
data = pd.read_csv('../training.1600000.processed.noemoticon.csv', encoding="ISO-8859-1", skiprows = new_skip)
data.columns = ['target','id','date','flag','user','text']
data

,target,id,date,flag,user,text
0,0,1467821455,Mon Apr 06 22:22:32 PDT 2009,NO_QUERY,CiaraRenee,I need a hug
1,0,1467837819,Mon Apr 06 22:26:49 PDT 2009,NO_QUERY,SimoniNicole,would rather the first party send bad messages...
2,0,1467853356,Mon Apr 06 22:30:54 PDT 2009,NO_QUERY,dbmendel,Picked Mich St to win it all from the get go. ...
3,0,1467863716,Mon Apr 06 22:33:35 PDT 2009,NO_QUERY,stacyc37,sad that the 'feet' of my macbook just fell off
4,0,1467878983,Mon Apr 06 22:37:41 PDT 2009,NO_QUERY,dancingmanikin,poor socks luvvvvv the golden retriever!! I w...
...,...,...,...,...,...,...
29995,4,2193528780,Tue Jun 16 08:34:51 PDT 2009,NO_QUERY,11chiqa11,@The_Real_Tevin ei how u doin today?
29996,4,2193552007,Tue Jun 16 08:36:44 PDT 2009,NO_QUERY,Bellaleyla,"@Sky_1995 sorryyy. i gotta go now, my mom want..."
29997,4,2193554492,Tue Jun 16 08:36:57 PDT 2009,NO_QUERY,scottkoegler,@PaulMiller Do that! I appreciate the calories...
29998,4,2193577154,Tue Jun 16 08:38:49 PDT 2009,NO_QUERY,kcnitt,"oh yes, and btw, 8.00"


### Prepare Textual Data for Sentiment Analysis

Now, apply the functions you have written in Challenge 1 to your whole data set. These functions include:

* `clean_up()`

* `tokenize()`

* `stem_and_lemmatize()`

* `remove_stopwords()`

Create a new column called `text_processed` in the dataframe to contain the processed data. At the end, your `text_processed` column should contain lists of word tokens that are cleaned up. Your data should look like below:

![Processed Data](data-cleaning-results.png)

In [4]:
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt
import seaborn as sns
import re
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt
import seaborn as sns
import re
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer, WordNetLemmatizer
import nltk
from nltk.corpus import stopwords

In [5]:
def clean_up(s):
    """
    Cleans up numbers, URLs, and special characters from a string.

    Args:
        s: The string to be cleaned up.

    Returns:
        A string that has been cleaned up.
    """
    clean_s = s
    
    clean_s = clean_s.lower()
    clean_s = re.sub(r'(http|https|ftp|ssh)://([\w_-]+(?:(?:\.[\w_-]+)+))([\w.,@?^=%&:/~+#-]*[\w@?^=%&/~+#-])?', ' ' , clean_s) 
    clean_s = BeautifulSoup(clean_s, 'lxml').get_text().strip() 
    clean_s = re.sub(r'\W+',' ',clean_s) # Replace everything non-alphanumeric by ' '
    clean_s = re.sub(r'_\+', ' ', clean_s) # Replace everything non-alphanumeric by ' '
    clean_s = re.sub(r'\s+',' ',clean_s) # Replace one or more whitespaces by  ' '
    clean_s = re.sub(r'\d+',' ',clean_s) # Replace one or more digits by  ' '
        
    return clean_s

In [6]:
data['processed_text'] = data['text'].apply(clean_up)

C:\Users\joana\anaconda3\lib\site-packages\bs4\__init__.py:435: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  warnings.warn(


In [7]:
data

,target,id,date,flag,user,text,processed_text
0,0,1467821455,Mon Apr 06 22:22:32 PDT 2009,NO_QUERY,CiaraRenee,I need a hug,i need a hug
1,0,1467837819,Mon Apr 06 22:26:49 PDT 2009,NO_QUERY,SimoniNicole,would rather the first party send bad messages...,would rather the first party send bad messages...
2,0,1467853356,Mon Apr 06 22:30:54 PDT 2009,NO_QUERY,dbmendel,Picked Mich St to win it all from the get go. ...,picked mich st to win it all from the get go w...
3,0,1467863716,Mon Apr 06 22:33:35 PDT 2009,NO_QUERY,stacyc37,sad that the 'feet' of my macbook just fell off,sad that the feet of my macbook just fell off
4,0,1467878983,Mon Apr 06 22:37:41 PDT 2009,NO_QUERY,dancingmanikin,poor socks luvvvvv the golden retriever!! I w...,poor socks luvvvvv the golden retriever i want...
...,...,...,...,...,...,...,...
29995,4,2193528780,Tue Jun 16 08:34:51 PDT 2009,NO_QUERY,11chiqa11,@The_Real_Tevin ei how u doin today?,the_real_tevin ei how u doin today
29996,4,2193552007,Tue Jun 16 08:36:44 PDT 2009,NO_QUERY,Bellaleyla,"@Sky_1995 sorryyy. i gotta go now, my mom want...",sky_ sorryyy i gotta go now my mom want to g...
29997,4,2193554492,Tue Jun 16 08:36:57 PDT 2009,NO_QUERY,scottkoegler,@PaulMiller Do that! I appreciate the calories...,paulmiller do that i appreciate the calories ...
29998,4,2193577154,Tue Jun 16 08:38:49 PDT 2009,NO_QUERY,kcnitt,"oh yes, and btw, 8.00",oh yes and btw


In [8]:
def tokenize(s):
    """
    Tokenize a string.

    Args:
        s: String to be tokenized.

    Returns:
        A list of words as the result of tokenization.
    """
    token_s = word_tokenize(s)
    return token_s

In [9]:
data['processed_text'] = data['processed_text'].apply(tokenize)
data

,target,id,date,flag,user,text,processed_text
0,0,1467821455,Mon Apr 06 22:22:32 PDT 2009,NO_QUERY,CiaraRenee,I need a hug,"[i, need, a, hug]"
1,0,1467837819,Mon Apr 06 22:26:49 PDT 2009,NO_QUERY,SimoniNicole,would rather the first party send bad messages...,"[would, rather, the, first, party, send, bad, ..."
2,0,1467853356,Mon Apr 06 22:30:54 PDT 2009,NO_QUERY,dbmendel,Picked Mich St to win it all from the get go. ...,"[picked, mich, st, to, win, it, all, from, the..."
3,0,1467863716,Mon Apr 06 22:33:35 PDT 2009,NO_QUERY,stacyc37,sad that the 'feet' of my macbook just fell off,"[sad, that, the, feet, of, my, macbook, just, ..."
4,0,1467878983,Mon Apr 06 22:37:41 PDT 2009,NO_QUERY,dancingmanikin,poor socks luvvvvv the golden retriever!! I w...,"[poor, socks, luvvvvv, the, golden, retriever,..."
...,...,...,...,...,...,...,...
29995,4,2193528780,Tue Jun 16 08:34:51 PDT 2009,NO_QUERY,11chiqa11,@The_Real_Tevin ei how u doin today?,"[the_real_tevin, ei, how, u, doin, today]"
29996,4,2193552007,Tue Jun 16 08:36:44 PDT 2009,NO_QUERY,Bellaleyla,"@Sky_1995 sorryyy. i gotta go now, my mom want...","[sky_, sorryyy, i, got, ta, go, now, my, mom, ..."
29997,4,2193554492,Tue Jun 16 08:36:57 PDT 2009,NO_QUERY,scottkoegler,@PaulMiller Do that! I appreciate the calories...,"[paulmiller, do, that, i, appreciate, the, cal..."
29998,4,2193577154,Tue Jun 16 08:38:49 PDT 2009,NO_QUERY,kcnitt,"oh yes, and btw, 8.00","[oh, yes, and, btw]"


In [10]:
def stem_and_lemmatize(l):
    """
    Perform stemming and lemmatization on a list of words.

    Args:
        l: A list of strings.

    Returns:
        A list of strings after being stemmed and lemmatized.
    """

    port = PorterStemmer()
    stem = [port.stem(word) for word in l]

    lem = WordNetLemmatizer()

    stem_lem = [lem.lemmatize(word) for word in stem]
    
    return stem_lem

In [11]:
data['processed_text'] = data['processed_text'].apply(stem_and_lemmatize)

In [12]:
def remove_stopwords(l):
    """
    Remove English stopwords from a list of strings.

    Args:
        l: A list of strings.

    Returns:
        A list of strings after stop words are removed.
    """
    stop_words = list(stopwords.words('english'))
    
    new_list = [word for word in l if word not in stop_words]
    return new_list

In [13]:
data['processed_text'] = data['processed_text'].apply(remove_stopwords)

In [14]:
data

,target,id,date,flag,user,text,processed_text
0,0,1467821455,Mon Apr 06 22:22:32 PDT 2009,NO_QUERY,CiaraRenee,I need a hug,"[need, hug]"
1,0,1467837819,Mon Apr 06 22:26:49 PDT 2009,NO_QUERY,SimoniNicole,would rather the first party send bad messages...,"[would, rather, first, parti, send, bad, messa..."
2,0,1467853356,Mon Apr 06 22:30:54 PDT 2009,NO_QUERY,dbmendel,Picked Mich St to win it all from the get go. ...,"[pick, mich, st, win, get, go, wa, feel, prett..."
3,0,1467863716,Mon Apr 06 22:33:35 PDT 2009,NO_QUERY,stacyc37,sad that the 'feet' of my macbook just fell off,"[sad, foot, macbook, fell]"
4,0,1467878983,Mon Apr 06 22:37:41 PDT 2009,NO_QUERY,dancingmanikin,poor socks luvvvvv the golden retriever!! I w...,"[poor, sock, luvvvvv, golden, retriev, want, o..."
...,...,...,...,...,...,...,...
29995,4,2193528780,Tue Jun 16 08:34:51 PDT 2009,NO_QUERY,11chiqa11,@The_Real_Tevin ei how u doin today?,"[the_real_tevin, ei, u, doin, today]"
29996,4,2193552007,Tue Jun 16 08:36:44 PDT 2009,NO_QUERY,Bellaleyla,"@Sky_1995 sorryyy. i gotta go now, my mom want...","[sky_, sorryyi, got, ta, go, mom, want, go, th..."
29997,4,2193554492,Tue Jun 16 08:36:57 PDT 2009,NO_QUERY,scottkoegler,@PaulMiller Do that! I appreciate the calories...,"[paulmil, appreci, calori, miss]"
29998,4,2193577154,Tue Jun 16 08:38:49 PDT 2009,NO_QUERY,kcnitt,"oh yes, and btw, 8.00","[oh, ye, btw]"


### Creating Bag of Words

The purpose of this step is to create a [bag of words](https://en.wikipedia.org/wiki/Bag-of-words_model) from the processed data. The bag of words contains all the unique words in your whole text body (a.k.a. *corpus*) with the number of occurrence of each word. It will allow you to understand which words are the most important features across the whole corpus.

Also, you can imagine you will have a massive set of words. The less important words (i.e. those of very low number of occurrence) do not contribute much to the sentiment. Therefore, you only need to use the most important words to build your feature set in the next step. In our case, we will use the top 5,000 words with the highest frequency to build the features.

In the cell below, combine all the words in `text_processed` and calculate the frequency distribution of all words. A convenient library to calculate the term frequency distribution is NLTK's `FreqDist` class ([documentation](https://www.nltk.org/api/nltk.html#module-nltk.probability)). Then select the top 5,000 words from the frequency distribution.

In [15]:
from nltk.probability import FreqDist

In [16]:
bag_words =  nltk.FreqDist(data['processed_text'].explode().to_list())
bag_words

FreqDist({'go': 2606, 'get': 2081, 'day': 1985, 'wa': 1954, 'thi': 1743, 'good': 1723, 'love': 1580, 'work': 1574, 'like': 1536, 'u': 1378, ...})

In [17]:
bag_words = sorted(bag_words.items(), key=lambda x:x[1], reverse=True)[:5000]
print(bag_words)

[('go', 2606), ('get', 2081), ('day', 1985), ('wa', 1954), ('thi', 1743), ('good', 1723), ('love', 1580), ('work', 1574), ('like', 1536), ('u', 1378), ('today', 1333), ('got', 1314), ('time', 1205), ('thank', 1130), ('miss', 1104), ('lol', 1094), ('one', 1076), ('back', 1076), ('know', 1040), ('want', 1024), ('feel', 1002), ('see', 994), ('im', 980), ('think', 967), ('realli', 902), ('still', 860), ('night', 851), ('hope', 847), ('well', 842), ('need', 830), ('na', 811), ('new', 799), ('watch', 789), ('oh', 780), ('make', 772), ('look', 735), ('much', 711), ('come', 703), ('home', 698), ('ha', 694), ('last', 693), ('morn', 676), ('twitter', 671), ('tomorrow', 669), ('wish', 613), ('great', 604), ('wait', 591), ('sleep', 585), ('haha', 566), ('sad', 559), ('bad', 549), ('onli', 535), ('fun', 529), ('right', 514), ('happi', 514), ('tri', 512), ('follow', 509), ('whi', 508), ('friend', 505), ('week', 503), ('thing', 493), ('veri', 493), ('hi', 491), ('would', 485), ('tonight', 484), ('sor

In [18]:
bag_list = []
for n in range(0,len(bag_words)):
    bag_list.append(bag_words[n][0])

In [19]:
bag_list

['go',
 'get',
 'day',
 'wa',
 'thi',
 'good',
 'love',
 'work',
 'like',
 'u',
 'today',
 'got',
 'time',
 'thank',
 'miss',
 'lol',
 'one',
 'back',
 'know',
 'want',
 'feel',
 'see',
 'im',
 'think',
 'realli',
 'still',
 'night',
 'hope',
 'well',
 'need',
 'na',
 'new',
 'watch',
 'oh',
 'make',
 'look',
 'much',
 'come',
 'home',
 'ha',
 'last',
 'morn',
 'twitter',
 'tomorrow',
 'wish',
 'great',
 'wait',
 'sleep',
 'haha',
 'sad',
 'bad',
 'onli',
 'fun',
 'right',
 'happi',
 'tri',
 'follow',
 'whi',
 'friend',
 'week',
 'thing',
 'veri',
 'hi',
 'would',
 'tonight',
 'sorri',
 'take',
 'though',
 'gon',
 'say',
 'way',
 'better',
 'nice',
 'even',
 'yeah',
 'hour',
 'hate',
 'peopl',
 'could',
 'tweet',
 'weekend',
 'awesom',
 'show',
 'bed',
 'school',
 'hey',
 'play',
 'start',
 'ye',
 'let',
 'final',
 'never',
 'guy',
 'use',
 'littl',
 'life',
 'ok',
 'wan',
 'year',
 'long',
 'next',
 'soon',
 'rain',
 'suck',
 'first',
 'sick',
 'tire',
 'cant',
 'movi',
 'everyon',
 '

### Building Features

Now let's build the features. Using the top 5,000 words, create a 2-dimensional matrix to record whether each of those words is contained in each document (tweet). Then you also have an output column to indicate whether the sentiment in each tweet is positive. For example, assuming your bag of words has 5 items (`['one', 'two', 'three', 'four', 'five']`) out of 4 documents (`['A', 'B', 'C', 'D']`), your feature set is essentially:

| Doc | one | two | three | four | five | is_positive |
|---|---|---|---|---|---|---|
| A | True | False | False | True | False | True |
| B | False | False | False | True | True | False |
| C | False | True | False | False | False | True |
| D | True | False | False | False | True | False|

However, because the `nltk.NaiveBayesClassifier.train` class we will use in the next step does not work with Pandas dataframe, the structure of your feature set should be converted to the Python list looking like below:

```python
[
	({
		'one': True,
		'two': False,
		'three': False,
		'four': True,
		'five': False
	}, True),
	({
		'one': False,
		'two': False,
		'three': False,
		'four': True,
		'five': True
	}, False),
	({
		'one': False,
		'two': True,
		'three': False,
		'four': False,
		'five': False
	}, True),
	({
		'one': True,
		'two': False,
		'three': False,
		'four': False,
		'five': True
	}, False)
]
```

To help you in this step, watch the [following video](https://www.youtube.com/watch?v=-vVskDsHcVc) to learn how to build the feature set with Python and NLTK. The source code in this video can be found [here](https://pythonprogramming.net/words-as-features-nltk-tutorial/).

[![Building Features](building-features.jpg)](https://www.youtube.com/watch?v=-vVskDsHcVc)

In [29]:
def find_features(document):
   # words = set(document)
    features = {}
    for w in bag_list:
        features[w] = (w in document)
    return features# your code here

In [45]:
feature_set = [(find_features(document), bool(target)) for (text, target) in zip(data['processed_text'], data['target'])]

In [32]:
#feature_set = find_features(data['processed_text'])

In [35]:
#feature_set = [ (find_features(document)) for document in data['processed_text'] ]

In [50]:
feature_set[3]

({'go': False,
  'get': False,
  'day': False,
  'wa': False,
  'thi': False,
  'good': False,
  'love': False,
  'work': False,
  'like': False,
  'u': False,
  'today': False,
  'got': False,
  'time': False,
  'thank': False,
  'miss': False,
  'lol': False,
  'one': False,
  'back': False,
  'know': False,
  'want': False,
  'feel': False,
  'see': False,
  'im': False,
  'think': False,
  'realli': False,
  'still': False,
  'night': False,
  'hope': False,
  'well': False,
  'need': True,
  'na': False,
  'new': False,
  'watch': False,
  'oh': False,
  'make': False,
  'look': False,
  'much': False,
  'come': False,
  'home': False,
  'ha': False,
  'last': False,
  'morn': False,
  'twitter': False,
  'tomorrow': False,
  'wish': False,
  'great': False,
  'wait': False,
  'sleep': False,
  'haha': False,
  'sad': False,
  'bad': False,
  'onli': False,
  'fun': False,
  'right': False,
  'happi': False,
  'tri': False,
  'follow': False,
  'whi': False,
  'friend': False,
  '

### Building and Traininng Naive Bayes Model

In this step you will split your feature set into a training and a test set. Then you will create a Bayes classifier instance using `nltk.NaiveBayesClassifier.train` ([example](https://www.nltk.org/book/ch06.html)) to train with the training dataset.

After training the model, call `classifier.show_most_informative_features()` to inspect the most important features. The output will look like:

```
Most Informative Features
	    snow = True            False : True   =     34.3 : 1.0
	  easter = True            False : True   =     26.2 : 1.0
	 headach = True            False : True   =     20.9 : 1.0
	    argh = True            False : True   =     17.6 : 1.0
	unfortun = True            False : True   =     16.9 : 1.0
	    jona = True             True : False  =     16.2 : 1.0
	     ach = True            False : True   =     14.9 : 1.0
	     sad = True            False : True   =     13.0 : 1.0
	  parent = True            False : True   =     12.9 : 1.0
	  spring = True            False : True   =     12.7 : 1.0
```

The [following video](https://www.youtube.com/watch?v=rISOsUaTrO4) will help you complete this step. The source code in this video can be found [here](https://pythonprogramming.net/naive-bayes-classifier-nltk-tutorial/).

[![Building and Training NB](nb-model-building.jpg)](https://www.youtube.com/watch?v=rISOsUaTrO4)

In [33]:
from nltk.classify.api import ClassifierI

In [46]:
# set that we'll train our classifier with
training_set = feature_set[:200]

# set that we'll test against.
testing_set = feature_set[2000:]

In [47]:
classifier = nltk.NaiveBayesClassifier.train(training_set)

In [55]:
classifier.show_most_informative_features()

### Testing Naive Bayes Model

Now we'll test our classifier with the test dataset. This is done by calling `nltk.classify.accuracy(classifier, test)`.

As mentioned in one of the tutorial videos, a Naive Bayes model is considered OK if your accuracy score is over 0.6. If your accuracy score is over 0.7, you've done a great job!

In [49]:
print("Classifier accuracy percent:",(nltk.classify.accuracy(classifier, testing_set))*100)

Classifier accuracy percent: 46.42857142857143


## Bonus Question 1: Improve Model Performance

If you are still not exhausted so far and want to dig deeper, try to improve your classifier performance. There are many aspects you can dig into, for example:

* Improve stemming and lemmatization. Inspect your bag of words and the most important features. Are there any words you should furuther remove from analysis? You can append these words to further remove to the stop words list.

* Remember we only used the top 5,000 features to build model? Try using different numbers of top features. The bottom line is to use as few features as you can without compromising your model performance. The fewer features you select into your model, the faster your model is trained. Then you can use a larger sample size to improve your model accuracy score.

In [ ]:
# your code here

## Bonus Question 2: Machine Learning Pipeline

In a new Jupyter Notebook, combine all your codes into a function (or a class). Your new function will execute the complete machine learning pipeline job by receiving the dataset location and output the classifier. This will allow you to use your function to predict the sentiment of any tweet in real time. 

In [ ]:
# your code here

## Bonus Question 3: Apache Spark

If you have completed the Apache Spark advanced topic lab, what you can do is to migrate your pipeline from local to a Databricks Notebook. Share your notebook with your instructor and classmates to show off your achievements!

In [ ]:
# your code here